# 要約 
このJupyter Notebookは、Kaggleの「LMSYS - Chatbot Arena」競技において、ユーザーの好みを予測するために設計されています。具体的には、与えられた2つの応答（モデルAとモデルB）から、ユーザーがどちらの応答を好むかを予測するための機械学習モデルを構築します。この競技の目標は、応答の好みを正確に予測するための「報酬モデル」を作成することです。

### 問題の概要
Notebookでは、チャットボットからの異なる応答（レスポンスAとレスポンスB）が与えられ、それらがユーザーによってどのように評価されるかを予測するためのモデルを構築しています。データは、プロンプトとそれに対する2つの応答、およびそれに関連するラベルから構成されています。

### 使用されている手法とライブラリ
このノートブックでは、以下の手法とライブラリが使用されています：

- **ライブラリ**:
  - `Keras`とその関連ライブラリ`keras_nlp`: 深層学習モデルの構築に使用。
  - `TensorFlow`: モデルのトレーニングと推論に使用。
  - `pandas`: データの読み込みと処理に使用。
  - `scikit-learn`: データの分割に使用。
  - `tqdm`: プロセス中の進捗バー表示に使用。
  - `transformers`: 事前訓練された言語モデルを利用するために使用（このコードでは特にDeBERTa V3が利用されています）。

- **手法**:
  - **データ処理**: JSONデータを読み込んでプロンプトと応答を結合し、クラスラベルにマッピングします。
  - **データセットの構築**: TensorFlowのデータセットを使用して効率的にバッチ処理を行います。
  - **モデルのアーキテクチャ**: DeBERTa V3をバックボーンとして使用し、応答AとBの埋め込みを結合して最終的な予測を行います。
  - **モデルのトレーニング**: 渡されたデータセットを使用し、ロスが最小のモデルを選択するよう設定されています。

### 結果の出力
トレーニングが完了したら、最良のモデルの重みが読み込まれ、テストデータに対する予測が行われます。予測結果はCSVファイルとして保存され、最終的にKaggleへ提出する形式に整理されています。

このNotebookは、効果的なデータ処理とモデルの構築を通じて、ユーザーの応答選好を予測するための実践的なアプローチを示しています。

---


# 用語概説 
以下は、提供されたJupyter Notebookに見られる専門用語の解説リストです。初心者がつまずきやすいマイナーな用語や、特有のドメイン知識に焦点を当てています。

1. **Keras**: Pythonの高水準なニューラルネットワークAPIで、TensorFlowの上に構築されています。簡潔かつ使いやすく、深層学習モデルを迅速に構築、トレーニング、評価するために使用されます。

2. **KerasNLP**: Keras用の自然言語処理ライブラリで、最新のNLPモデルやトークナイザー、データセットの処理を簡略化します。特にTransformerアーキテクチャに特化した機能が含まれています。

3. **bfloat16**: 「brain floating point format」と呼ばれる数値データ型で、主に深層学習で使用されます。精度を犠牲にすることなく、計算のメモリ使用量を削減する利点があります。

4. **層化サンプリング (Stratified Sampling)**: データセットを分割するときに、各クラスの比率を維持する方法です。クラスの不均衡を防ぎ、モデルの性能を向上させるために重要です。

5. **LoRA (Low-Rank Adaptation)**: 既存のモデルのパラメータを低次元のサブスペースで最適化する手法で、特にメモリ効率を高めつつファインチューニングを行う際に用いられます。

6. **パディング (Padding)**: データのシーケンス長を揃えるために、短いシーケンスの末尾にゼロまたは特定の値を加える操作です。自然言語処理モデルでは、バッチ内の全てのシーケンスを同じ長さに保つためによく使用されます。

7. **オプティマイザー (Optimizer)**: モデルが学習する際、損失関数を最小化するために使用するアルゴリズムのこと。AdamやSGDなどのさまざまなオプティマイザーが存在します。

8. **トークン化 (Tokenization)**: テキストを処理可能な単位（トークン）に分割する工程です。これは、自然言語モデルがテキストを理解しやすくするために必要です。

9. **カテゴリカルクロスエントロピー (Categorical Crossentropy)**: 多クラス分類問題において、モデルの予測した確率と実際のクラスラベルとの間の損失を計算する指標です。

10. **グローバル平均プーリング (Global Average Pooling)**: 特徴マップの各チャネルの平均値を計算する操作で、モデルの出力を低次元のベクトルに変換します。この方法は、過学習を防ぐのに役立ちます。

これらの用語は、特に初学者が機械学習や深層学習を学ぶ過程で出会う状況が多いので、理解しておくと良いでしょう。

---


In [ ]:
# このPython 3環境には、多くの便利な分析ライブラリがインストールされています
# これはkaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# 例えば、以下はいくつかの便利なパッケージの読み込み例です

import numpy as np # 線形代数用のライブラリ
import pandas as pd # データ処理用のライブラリ、CSVファイルの入出力（例: pd.read_csv）

# 入力データファイルは、読み取り専用の"../input/"ディレクトリで利用可能です
# 例えば、これを実行すると（実行ボタンをクリックするか、Shift+Enterを押すことで）入力ディレクトリ内のすべてのファイルをリストします

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    # 指定されたディレクトリ（dirname）内のファイルをすべて走査します
    for filename in filenames:
        # 各ファイルのフルパスを出力します
        print(os.path.join(dirname, filename))

# 現在のディレクトリ（/kaggle/working/）には最大20GBを書き込むことができ、
# 「すべてを保存して実行」したときに出力として保持されます
# また、/kaggle/temp/に一時ファイルを書き込むこともできますが、現在のセッション外では保存されません

In [ ]:
import os
# Kerasのバックエンドを設定します。"tensorflow"または"torch"のいずれかを指定できます
os.environ["KERAS_BACKEND"] = "tensorflow"  # ここではtensorflowを選択

import keras_nlp  # Kerasの自然言語処理用ライブラリをインポート
import keras  # Kerasライブラリをインポート
import tensorflow as tf  # TensorFlowライブラリをインポート

import numpy as np  # 数値計算用のライブラリをインポート
import pandas as pd  # データ処理用のライブラリをインポート
from tqdm import tqdm  # 進捗バーを表示するライブラリをインポート
import json  # JSONデータを扱うためのライブラリをインポート

import matplotlib.pyplot as plt  # グラフ描画用ライブラリをインポート
import matplotlib as mpl  # Matplotlibの設定を行うためのライブラリをインポート
import transformers  # Transformersライブラリをインポート（自然言語処理モデル用）

In [ ]:
# TensorFlow、Keras、KerasNLPのバージョンを表示します
print("TensorFlow:", tf.__version__)  # TensorFlowのバージョンを出力
print("Keras:", keras.__version__)  # Kerasのバージョンを出力
print("KerasNLP:", keras_nlp.__version__)  # KerasNLPのバージョンを出力

In [ ]:
# モデルの設定を保持するためのクラスを定義します
class CFG:
    seed = 50  # 乱数シードを指定します（再現性を保つため）
    sequence_length = 1024  # 入力シーケンスの長さを指定します
    epochs = 2  # 学習のエポック数を指定します
    batch_size = 2  # バッチサイズを指定します
    label2name = {0: 'winner_model_a', 1: 'winner_model_b', 2: 'winner_tie'}  
    # ラベルをモデル名にマッピングする辞書を定義します
    name2label = {v:k for k, v in label2name.items()}  
    # モデル名をラベルにマッピングする辞書を生成します
    class_labels = list(label2name.keys())  # クラスラベルのリストを生成します
    class_names = list(label2name.values())  # クラス名のリストを生成します

In [ ]:
# 乱数シードを設定して、再現性のある結果を得るための準備をします
keras.utils.set_random_seed(CFG.seed)  # Kerasの乱数シードを設定します

# 計算精度のポリシーを設定します。ここではbfloat16を使用します
keras.mixed_precision.set_global_policy("bfloat16")

# データセットのベースパスを定義します
BASE_PATH = '/kaggle/input/lmsys-chatbot-arena'  # データセットが格納されているパスを設定します

In [ ]:
# JSON形式のデータを読み込み、リストからNoneではない要素を結合して返す関数を定義します
def load_data(row):
    # 行データをJSONとして読み込みます
    row_list = json.loads(row)
    # Noneでない要素をスペースで結合して返します
    return " ".join(row for row in row_list if row != None)  # Noneでない要素を結合して1つの文字列にします

In [ ]:
# 学習データを読み込みます
df = pd.read_csv(f'{BASE_PATH}/train.csv')  # CSVファイルを読み込み、データフレームを作成します
# df = df.iloc[:100] # デモ用にデータのサブセットを使用する場合はこちらを有効にします

# 最初のプロンプトとそれに関連するレスポンスを取得します
df["prompt"] = df["prompt"].apply(load_data)  # プロンプト列にload_data関数を適用
df["response_a"] = df["response_a"].apply(load_data)  # レスポンスA列にload_data関数を適用
df["response_b"] = df["response_b"].apply(load_data)  # レスポンスB列にload_data関数を適用

# ラベルの変換を行います
df["class_name"] = df[["winner_model_a", "winner_model_b" , "winner_tie"]].idxmax(axis=1)  
# 各行の勝者モデルを判定し、対応するクラス名を取得
df["class_label"] = df.class_name.map(CFG.name2label)  # クラス名をクラスラベルにマッピング

# サンプルを表示します
df.head()  # データフレームの最初の数行を表示して確認します

In [ ]:
# テストデータを読み込みます
test_df = pd.read_csv(f'{BASE_PATH}/test.csv')  # テスト用のCSVファイルを読み込み、データフレームを作成します

# 最初のプロンプトとレスポンスを取得します
test_df["prompt"] = test_df["prompt"].apply(load_data)  # プロンプト列にload_data関数を適用
test_df["response_a"] = test_df["response_a"].apply(load_data)  # レスポンスA列にload_data関数を適用
test_df["response_b"] = test_df["response_b"].apply(load_data)  # レスポンスB列にload_data関数を適用

# サンプルを表示します
test_df.head()  # テストデータフレームの最初の数行を表示して確認します

In [ ]:
# オプション列を作成します。レスポンスAとレスポンスBをリストに格納します
df['options'] = df.apply(lambda row: [row.response_a, row.response_b], axis=1)
display(df.head(2))  # データフレームdfの最初の2行を表示します

# テストデータにも同様にオプション列を作成します
test_df['options'] = test_df.apply(lambda row: [row.response_a, row.response_b], axis=1)
display(test_df.head(2))  # テストデータフレームtest_dfの最初の2行を表示します

In [ ]:
from sklearn.model_selection import train_test_split  # パッケージをインポートします（データ分割用）

# データを訓練セットと検証セットに分割します
train_df, valid_df = train_test_split(df, test_size=0.1, stratify=df["class_label"])  
# 10%を検証セットとして分割し、クラスラベルに基づいて層化サンプリングを行います

In [ ]:
# DeBERTa V3の前処理器を設定します
preprocessor = keras_nlp.models.DebertaV3Preprocessor.from_preset(
    "deberta_v3_small_en",  # 使用するモデルのプリセット名を指定します
    sequence_length=CFG.sequence_length,  # 最大シーケンス長を設定します。短い場合はパディングされます
    dtype="bfloat16",  # データ型をbfloat16に設定します
)

In [ ]:
# 最初の行のオプションを前処理します
outs = preprocessor(df.options.iloc[0])  # 最初の行のレスポンスオプションを処理

# 各処理された出力の形状を表示します
for k, v in outs.items():
    print(k, ":", v.shape)  # キーと対応する出力の形状を表示します

In [ ]:
# テキストとラベルを前処理する関数を定義します
def preprocess_fn(text, label=None):
    text = preprocessor(text)  # テキストを前処理します
    # ラベルが存在する場合は、処理されたテキストとラベルを返します
    return (text, label) if label is not None else text  # ラベルがなければ処理されたテキストのみを返します

In [ ]:
# テキストとラベルからデータセットを構築する関数を定義します
def build_dataset(texts, labels=None, batch_size=32,
                  cache=True, shuffle=1024):
    AUTO = tf.data.AUTOTUNE  # AUTOTUNEオプションを設定
    # ラベルがない場合はテキストのみのスライスを作成し、ある場合はラベルをワンホットエンコーディングします
    slices = (texts,) if labels is None else (texts, keras.utils.to_categorical(labels, num_classes=3))  
    ds = tf.data.Dataset.from_tensor_slices(slices)  # スライスからデータセットを作成します
    ds = ds.cache() if cache else ds  # キャッシュを有効にする場合はデータセットをキャッシュ
    ds = ds.map(preprocess_fn, num_parallel_calls=AUTO)  # 前処理関数をマッピングします
    opt = tf.data.Options()  # データセットオプションを作成します
    if shuffle: 
        ds = ds.shuffle(shuffle, seed=CFG.seed)  # シャッフルが有効な場合、データセットをシャッフルします
        opt.experimental_deterministic = False  # 確定的でないオプションを設定
    ds = ds.with_options(opt)  # データセットオプションを設定します
    ds = ds.batch(batch_size, drop_remainder=False)  # バッチサイズでデータセットをバッチ化します
    ds = ds.prefetch(AUTO)  # 次のバッチを事前取得します
    return ds  # 構築したデータセットを返します

In [ ]:
# JAXライブラリをインポートします（コメントアウトされています）
# import jax
# GPUデバイスを取得します
# devices = jax.devices("gpu")
# デバイスを表示します
# print("devices", devices)

# Kerasのデータ並列処理を設定するためのオブジェクトを作成します（コメントアウトされています）
# data_parallel = keras.distribution.DataParallel(devices=devices)

In [ ]:
# データセットを取得する関数を定義します
def get_data():
    train_texts = train_df.options.tolist()  # 訓練用のテキストを抽出します
    train_labels = train_df.class_label.tolist()  # 訓練用のラベルを抽出します
    train_ds = build_dataset(train_texts, train_labels,
                             batch_size=CFG.batch_size,  # バッチサイズを設定
                             shuffle=True)  # 訓練データはシャッフルします

    # 検証データ
    valid_texts = valid_df.options.tolist()  # 検証用のテキストを抽出します
    valid_labels = valid_df.class_label.tolist()  # 検証用のラベルを抽出します
    valid_ds = build_dataset(valid_texts, valid_labels,
                             batch_size=CFG.batch_size,  # バッチサイズを設定
                             shuffle=False)  # 検証データはシャッフルしません

    # テストデータセットを構築します
    test_texts = test_df.options.tolist()  # テスト用のテキストを抽出します
    test_ds = build_dataset(test_texts,
                             batch_size=min(len(test_df), CFG.batch_size),  # テストデータのサイズに応じてバッチサイズを設定
                             shuffle=False)  # テストデータはシャッフルしません
    return train_ds, valid_ds, test_ds  # 訓練、検証、テストデータセットを返します

In [ ]:
# Kerasのデータ並列処理を設定します（コメントアウトされています）
# keras.distribution.set_distribution(data_parallel)  # モデルを両方のGPUで複製します

In [ ]:
# バックボーンモデルを取得する関数を定義します
def get_backbone():
    # DeBERTa V3のバックボーンモデルをプリセットから取得します
    backbone = keras_nlp.models.DebertaV3Backbone.from_preset(
        "deberta_v3_small_en",  # 使用するモデルのプリセット名を指定します
        dtype="bfloat16",  # データ型をbfloat16に設定します
    )
    backbone.enable_lora(8)  # LoRA（Low-Rank Adaptation）の有効化と設定を行います
    
    return backbone  # バックボーンモデルを返します

In [ ]:
# モデルのチェックポイントを設定するコールバックを定義します
ckpt_cb = keras.callbacks.ModelCheckpoint(f'best_model.weights.h5',  # モデルの重みを保存するファイル名
                                  monitor='val_log_loss',  # 検証ロスを監視
                                  save_best_only=True,  # 最良のモデルのみを保存
                                  save_weights_only=True,  # 重みのみを保存
                                  mode='min')  # 最小化を目指す設定（ロスが最小の時に保存）

In [ ]:
# カテゴリカルクロスエントロピーのメトリクスを定義します
log_loss = keras.metrics.CategoricalCrossentropy(name="log_loss")  # ロスの名前を"log_loss"と設定します

In [ ]:
# バックボーンを使用してモデルを構築する関数を定義します
def get_model(backbone):
    # 入力層を定義します
    inputs = {
        "token_ids": keras.Input(shape=(2, None), dtype=tf.int16, name="token_ids"),  # トークンID入力
        "padding_mask": keras.Input(shape=(2, None), dtype=tf.int16, name="padding_mask"),  # パディングマスクリスト
    }

    # 最初のレスポンス（Response A）に対する埋め込みを計算します: (P + R_A)
    response_a = {k: v[:, 0, :] for k, v in inputs.items()}  # 入力からAのレスポンスを抽出
    embed_a = backbone(response_a)  # バックボーンを通して埋め込みを計算

    # 2番目のレスポンス（Response B）に対する埋め込みを計算します: (P + R_B)
    response_b = {k: v[:, 1, :] for k, v in inputs.items()}  # 入力からBのレスポンスを抽出
    embed_b = backbone(response_b)  # 同じバックボーンを使用して埋め込みを計算

    # 最終出力を計算します
    embeds = keras.layers.Concatenate(axis=-1)([embed_a, embed_b])  # AとBの埋め込みを結合
    embeds = keras.layers.GlobalAveragePooling1D()(embeds)  # グローバル平均プーリングを適用
    outputs = keras.layers.Dense(3, activation="softmax", name="classifier")(embeds)  # 出力層を定義（3クラス分類）

    # モデルを定義
    model = keras.Model(inputs, outputs)

    # 最適化アルゴリズム、損失、メトリクスを指定してモデルをコンパイル
    model.compile(
        optimizer=keras.optimizers.Adam(5e-6),  # Adamオプティマイザを使用
        loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.02),  # カテゴリカルクロスエントロピーを損失関数とする
        metrics=[
            log_loss,  # ロスメトリクス
            keras.metrics.CategoricalAccuracy(name="accuracy"),  # カテゴリカル精度メトリクス
        ],
    )
    
    return model  # 構築したモデルを返します

In [ ]:
# 分散処理のための戦略を設定します
strategy = tf.distribute.MirroredStrategy()  # 複数のGPUでの計算を有効にします

# 戦略スコープをオープンします
with strategy.scope():
    
    train_ds, valid_ds, test_ds = get_data()  # データセットを取得します
    
    backbone = get_backbone()  # バックボーンモデルを取得します
    
    model = get_model(backbone)  # モデルを構築します
    
    # モデルを訓練します
    history = model.fit(
        train_ds,  # 訓練データ
        epochs=CFG.epochs,  # 訓練のエポック数
        validation_data=valid_ds,  # 検証データ
        callbacks=[ckpt_cb]  # モデルチェックポイントコールバック
    )

In [ ]:
# 最良のモデルの重みを読み込みます
model.load_weights('/kaggle/working/best_model.weights.h5')  # 保存したモデルの重みを指定したパスから読み込みます

In [ ]:
# 訓練済みモデルを使用してテストデータに対する予測を行います
test_preds = model.predict(test_ds, verbose=1)  # テストデータに対する予測を実行し、進捗を表示します

In [ ]:
# 提出用のデータフレームを作成します
sub_df = test_df[["id"]].copy()  # テストデータからID列をコピーします
sub_df[CFG.class_names] = test_preds.tolist()  # 予測結果をデータフレームに追加します
sub_df.to_csv("submission.csv", index=False)  # 提出用のCSVファイルとして保存します
sub_df.head()  # 作成したデータフレームの最初の数行を表示して確認します

# 📌 | 参考文献

* [LLM Science Exam: KerasCore + KerasNLP [TPU]](https://www.kaggle.com/code/awsaf49/llm-science-exam-kerascore-kerasnlp-tpu)
* [AES 2.0: KerasNLPスターター](https://www.kaggle.com/code/awsaf49/aes-2-0-kerasnlp-starter)
* [LMSYS: KerasNLPスターター](https://www.kaggle.com/code/awsaf49/lmsys-kerasnlp-starter)